#### Importando bibliotecas

In [1]:
from pymongo import MongoClient
import requests
import pandas as pd

#### Conectando ao MongoDB Atlas

In [ ]:
# Conectar ao MongoDB Atlas
client = MongoClient("")
# Seleciona o banco de dados
db = client[""]
# Seleciona a coleção
collection = db["desemprego"]

In [3]:
# Recuperar dados da coleção
data = list(collection.find())
data

[]

In [4]:
# Converter para DataFrame
df = pd.DataFrame(data)
print(df)

Empty DataFrame
Columns: []
Index: []


#### Consumindo API e enviando dados para o banco

In [5]:
# Endpoint da API do IBGE para o agregado de taxa de desocupação
url = "https://servicodados.ibge.gov.br/api/v3/agregados/4099/periodos/201201-202403/variaveis/4099?localidades=N3[all]"

# Realizar a requisição
response = requests.get(url)

# Verificar se a requisição foi bem-sucedida
if response.status_code == 200:
    data = response.json()

    # Extrair e organizar os dados
    desemprego_data = []
    for resultado in data:
        for serie in resultado["resultados"][0]["series"]:
            estado_nome = serie["localidade"]["nome"]  # Obtém o nome do estado
            for periodo, valor in serie["serie"].items():
                desemprego_data.append({
                    "periodo": periodo,
                    "desemprego": valor,
                    "estado": estado_nome
                })
else:
    print("Erro:", response.status_code, response.text)

In [6]:
# Converter para DataFrame
df = pd.DataFrame(desemprego_data)
print(df)

     periodo desemprego            estado
0     201201        8.1          Rondônia
1     201202        6.3          Rondônia
2     201203        5.9          Rondônia
3     201204        5.2          Rondônia
4     201301        6.2          Rondônia
...      ...        ...               ...
1372  202303        8.8  Distrito Federal
1373  202304        9.6  Distrito Federal
1374  202401        9.5  Distrito Federal
1375  202402        9.7  Distrito Federal
1376  202403        8.8  Distrito Federal

[1377 rows x 3 columns]


In [7]:
#tranformar dados em dicionário para insertir no banco de dados
data_dict = df.to_dict("records")
print(data_dict)

[{'periodo': '201201', 'desemprego': '8.1', 'estado': 'Rondônia'}, {'periodo': '201202', 'desemprego': '6.3', 'estado': 'Rondônia'}, {'periodo': '201203', 'desemprego': '5.9', 'estado': 'Rondônia'}, {'periodo': '201204', 'desemprego': '5.2', 'estado': 'Rondônia'}, {'periodo': '201301', 'desemprego': '6.2', 'estado': 'Rondônia'}, {'periodo': '201302', 'desemprego': '4.9', 'estado': 'Rondônia'}, {'periodo': '201303', 'desemprego': '4.6', 'estado': 'Rondônia'}, {'periodo': '201304', 'desemprego': '5.0', 'estado': 'Rondônia'}, {'periodo': '201401', 'desemprego': '5.0', 'estado': 'Rondônia'}, {'periodo': '201402', 'desemprego': '4.2', 'estado': 'Rondônia'}, {'periodo': '201403', 'desemprego': '4.2', 'estado': 'Rondônia'}, {'periodo': '201404', 'desemprego': '3.6', 'estado': 'Rondônia'}, {'periodo': '201501', 'desemprego': '4.5', 'estado': 'Rondônia'}, {'periodo': '201502', 'desemprego': '4.9', 'estado': 'Rondônia'}, {'periodo': '201503', 'desemprego': '6.7', 'estado': 'Rondônia'}, {'periodo

In [8]:
# Inserindo os dados no MongoDB
collection.insert_many(data_dict)

InsertManyResult([ObjectId('674b97bebe63e90c45031c58'), ObjectId('674b97bebe63e90c45031c59'), ObjectId('674b97bebe63e90c45031c5a'), ObjectId('674b97bebe63e90c45031c5b'), ObjectId('674b97bebe63e90c45031c5c'), ObjectId('674b97bebe63e90c45031c5d'), ObjectId('674b97bebe63e90c45031c5e'), ObjectId('674b97bebe63e90c45031c5f'), ObjectId('674b97bebe63e90c45031c60'), ObjectId('674b97bebe63e90c45031c61'), ObjectId('674b97bebe63e90c45031c62'), ObjectId('674b97bebe63e90c45031c63'), ObjectId('674b97bebe63e90c45031c64'), ObjectId('674b97bebe63e90c45031c65'), ObjectId('674b97bebe63e90c45031c66'), ObjectId('674b97bebe63e90c45031c67'), ObjectId('674b97bebe63e90c45031c68'), ObjectId('674b97bebe63e90c45031c69'), ObjectId('674b97bebe63e90c45031c6a'), ObjectId('674b97bebe63e90c45031c6b'), ObjectId('674b97bebe63e90c45031c6c'), ObjectId('674b97bebe63e90c45031c6d'), ObjectId('674b97bebe63e90c45031c6e'), ObjectId('674b97bebe63e90c45031c6f'), ObjectId('674b97bebe63e90c45031c70'), ObjectId('674b97bebe63e90c45031c